<a href="https://colab.research.google.com/github/skkzsh/llm-notebook/blob/main/langchain_summarize_epub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# これは何？
LangChainに本 (EPub) を要約してもらいます.

# 準備

まずモジュールをインストールします.

In [ ]:
!pip install openai tiktoken langchain unstructured

# EPub
ココでは例として [process_book.epub](https://github.com/Shinpeim/process-book/tree/master/release) を使います.  
Google Driveにダウンロードしてください.  

Google DriveをColabにmountします.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

EPubファイルを配置したパスを変数として定義しておきます.

In [ ]:
epub = '/content/drive/MyDrive/process_book.epub'

# EPubをロード
[`UnstructuredEPubLoader`](https://python.langchain.com/en/latest/modules/indexes/document_loaders/examples/epub.html) で, EPubファイルをロードできます.

In [ ]:
%%time
from langchain.document_loaders import UnstructuredEPubLoader

loader = UnstructuredEPubLoader(epub)
docs = loader.load_and_split()

ロードした結果を出力すると, EPubファイルの内容が格納されていることが分かります.

In [ ]:
docs

分割されています.

In [ ]:
len(docs)

# 要約させる
`OPENAI_API_KEY` を設定します.

In [ ]:
%env OPENAI_API_KEY=

モデルには gpt-3.5-turbo を使います.

In [ ]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

[`load_summarize_chain`](https://python.langchain.com/en/latest/modules/chains/index_examples/summarize.html) で要約ができます.  
`chain_type` は `map_reduce` を指定してます.  
処理経過も確認するために `verbose` を `True` にしておきます (以降も同様).

In [ ]:
from langchain.chains.summarize import load_summarize_chain

summary_chain = load_summarize_chain(
    llm,
    chain_type="map_reduce",
    verbose=True,
)

要約を実行します.

In [ ]:
%%time
summary = summary_chain.run(docs)

要約結果を出力します.

In [ ]:
summary

しかし, 要約結果が英語で返ってきました.

# 日本語訳するプロンプトへ連鎖させる
そこで, 要約結果を日本語訳するプロンプトに渡すように実装します.  
まず, 日本語訳するプロンプトを作ります.

In [ ]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

template = """
以下の文章を日本語にしてください。
{text}
"""

prompt_template = PromptTemplate(
    template=template,
    input_variables=["text"],
)

ja_chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True,
)

[`SimpleSequentialChain`](https://python.langchain.com/en/latest/modules/chains/generic/sequential_chains.html) で, 要約するプロンプトの結果を, 日本語訳するプロンプトの入力として渡します.

In [ ]:
from langchain.chains import SimpleSequentialChain

ja_summary_chain = SimpleSequentialChain(
    chains=[summary_chain, ja_chain],
    verbose=True,
)

要約を実行します.

In [ ]:
%%time
ja_summary = ja_summary_chain.run(docs)

要約結果を出力します.

In [ ]:
ja_summary

要約結果が日本語で得られました.

# ライセンス
[![クリエイティブ・コモンズ・ライセンス](http://i.creativecommons.org/l/by-sa/3.0/88x31.png)](http://creativecommons.org/licenses/by-sa/3.0/deed.ja)
このノートは[クリエイティブ・コモンズ 表示 - 継承 3.0 非移植 ライセンス](http://creativecommons.org/licenses/by-sa/3.0/deed.ja)の下で公開します.